# Jacobian of Four-Bar Mechanism 

The knee joint is actuated but the ankle joint is passive.  

## Todo's

  - consider $\theta_0$

In [3]:
import sympy as sp
sp.init_printing(use_latex='mathjax')

from sympy import symbols, sin, cos, sqrt, asin, atan


In [4]:
# Actuation (knee)
theta = symbols('theta')

# Passive joint (ankle) 
psi = symbols('psi')


In [5]:

# Constants defined by geometry.
theta0 = symbols('theta0')

shank, p1, p2, p3 = symbols('shank p1 p2 p3')



In [6]:
# Manually specified structure of equations. We first introduce variables based on the kinematic structure only.
# These can easily be precomputed in code. Here, they are just mentioned as a reference.

k1 = shank / p1
k2 = shank / p3
k3 = (shank**2 + p1**2 + p3**2 - p2**2 ) / (2 * p1 * p3)


# directly use these variables
k1, k2, k3 = symbols('k1 k2 k3')


In [10]:
from sympy import Eq, solve

#  k1 * cos(theta) - k2 * cos(psi) + k3 = cos(psi - theta)
#  k1 * cos(theta) - k2 * cos(0) + k3 = cos(0 - theta)
#  k1 * cos(theta) - k2 + k3 = cos( - theta)
#  k1 * cos(theta) - k2 + k3 = cos(theta)
#   - k2 + k3 = (1-k1) cos(theta)
#   (k3 - k2)/(1-k1) =  cos(theta)
# theta = arccos( (k3 - k2) / (1-k1) )

solve(eqn, theta)


⎡      ⎛                                ______________________________________
⎢      ⎜    2⋅ⅈ⋅ψ              ⅈ⋅ψ     ╱       2  2⋅ⅈ⋅ψ         3⋅ⅈ⋅ψ         
⎢      ⎜k₂⋅ℯ      + k₂ - 2⋅k₃⋅ℯ    - ╲╱  - 4⋅k₁ ⋅ℯ      + 4⋅k₁⋅ℯ      + 4⋅k₁⋅ℯ
⎢-ⅈ⋅log⎜──────────────────────────────────────────────────────────────────────
⎢      ⎜                                                                      
⎣      ⎝                                                                      

______________________________________________________________________________
ⅈ⋅ψ     2  4⋅ⅈ⋅ψ       2  2⋅ⅈ⋅ψ     2            3⋅ⅈ⋅ψ            ⅈ⋅ψ       2 
    + k₂ ⋅ℯ      + 2⋅k₂ ⋅ℯ      + k₂  - 4⋅k₂⋅k₃⋅ℯ      - 4⋅k₂⋅k₃⋅ℯ    + 4⋅k₃ ⋅
──────────────────────────────────────────────────────────────────────────────
       ⎛    ⅈ⋅ψ    ⎞                                                          
     2⋅⎝k₁⋅ℯ    - 1⎠                                                          

__________________⎞        ⎛                      

## Relationship between ankle ($\psi$) and knee ($\theta$)

In [8]:

# some helpers
cT = cos(theta)
sT = sin(theta)


A = k1 * cT + k2 + k3 + cT # C.34
B = -2*sT # C.35
C = k1 * cT - k2 + k3 - cT # C.36

D = sqrt(B * B - 4 * A * C)

D_fn = Function("D")(theta)

# ankle
psi = 2 * atan((-B + D_fn) / (2 * A)) #  C.39

psi = sp.simplify(psi)

psi_of_theta = psi



NameError: name 'Function' is not defined

In [ ]:
psi

      ⎛       D(θ)                 ⎞
      ⎜       ──── + sin(θ)        ⎟
      ⎜        2                   ⎟
2⋅atan⎜────────────────────────────⎟
      ⎝k₁⋅cos(θ) + k₂ + k₃ + cos(θ)⎠

### Derivatives

In [ ]:
from sympy import diff

dD_dtheta = diff(D, theta).simplify()

dD_dtheta




                  ⎛  2                    ⎞                      
                2⋅⎝k₁ ⋅cos(θ) + k₁⋅k₃ - k₂⎠⋅sin(θ)               
─────────────────────────────────────────────────────────────────
   ______________________________________________________________
  ╱     2    2                         2                   2     
╲╱  - k₁ ⋅cos (θ) - 2⋅k₁⋅k₃⋅cos(θ) + k₂  + 2⋅k₂⋅cos(θ) - k₃  + 1 

In [ ]:
from sympy import diff


dpsi_dtheta = diff(psi, theta)

D_fn = Function('D')(theta)

dpsi_dtheta = dpsi_dtheta.replace(D, D_fn)
dpsi_dtheta = dpsi_dtheta.simplify()

dpsi_dtheta

  ⎛                                    ⎛           d       ⎞                  
4⋅⎜(k₁ + 1)⋅(D(θ) + 2⋅sin(θ))⋅sin(θ) + ⎜2⋅cos(θ) + ──(D(θ))⎟⋅(k₁⋅cos(θ) + k₂ +
  ⎝                                    ⎝           dθ      ⎠                  
──────────────────────────────────────────────────────────────────────────────
                                    2                                   2     
                   (D(θ) + 2⋅sin(θ))  + 4⋅(k₁⋅cos(θ) + k₂ + k₃ + cos(θ))      

             ⎞
 k₃ + cos(θ))⎟
             ⎠
──────────────
              
              

In [ ]:
# We can implement the term dpsi_dtheta now in code. Therefore, we assume from now on that it is given.

from sympy import Function
psi = Function('psi')(theta)

## Coordinate systems

In [ ]:

from sympy.vector import CoordSys3D, AxisOrienter
from sympy import Matrix

# The reference system at the origin of the ankle. It is defined as follows:
# -z is the rotation axis of the joint
# x points upwards (away from the platform, aligned with the global z axis)
B = CoordSys3D('base')

orienter_psi = AxisOrienter(psi, -B.k )

# coordinate system of the ankle joint (after applying the joint rotation)
T_ankle = B.orient_new('ankle', (orienter_psi, ))

# helper coordinate system at the origin of the knee after applying the knee rotation
T_knee_base = T_ankle.locate_new('knee_base', shank * T_ankle.i )

T_knee_base



knee_base

In [ ]:

orienter_theta = AxisOrienter(theta, B.k )

# coordinate system of the knee after applying the rotation
T_knee = T_knee_base.orient_new('knee', (orienter_theta, ))

T_knee

knee

In [ ]:
# express the end effector in the knee joint coordinate system (T_knee). This captures the forward kinematics of child joints.
x, y, z, alpha, beta, gamma = symbols("x y z alpha beta gamma")

from sympy.vector import BodyOrienter, Point
euler = BodyOrienter(alpha, beta, gamma, rot_order="XYZ")

P_eef = T_knee.orient_new("EEF", euler, location=(T_knee.i * x + T_knee.j * y + T_knee.k * z))



In [ ]:
pos_vect = P_eef.origin.position_wrt(B)

# express the EEF in the base coordinate system (ankle base)
p_eef = pos_vect.to_matrix(B)

p_eef

# o = P_eef.rotation_matrix(B)

⎡shank⋅cos(ψ(θ)) + x⋅(sin(θ)⋅sin(ψ(θ)) + cos(θ)⋅cos(ψ(θ))) + y⋅(-sin(θ)⋅cos(ψ(
⎢                                                                             
⎢-shank⋅sin(ψ(θ)) + x⋅(sin(θ)⋅cos(ψ(θ)) - sin(ψ(θ))⋅cos(θ)) + y⋅(sin(θ)⋅sin(ψ(
⎢                                                                             
⎣                                                 z                           

θ)) + sin(ψ(θ))⋅cos(θ))⎤
                       ⎥
θ)) + cos(θ)⋅cos(ψ(θ)))⎥
                       ⎥
                       ⎦

# Jacobian

Pose of the end-effector
$$
p_{eef} = (x,y,z,\alpha, \beta, \gamma )^T
$$

The Jacobian $J \in \mathrm{R}^{6x1}$
$$
J = \frac{\partial p_{eef}}{\partial \theta}
$$


In [ ]:
from sympy import diff

# position part of the Jacobian
dp_dtheta = diff(p_eef, theta)
dp_dtheta = dp_dtheta.simplify()

dp_dtheta


⎡                  d            ⎛d           ⎞                   ⎛d           
⎢- shank⋅sin(ψ(θ))⋅──(ψ(θ)) + x⋅⎜──(ψ(θ)) - 1⎟⋅sin(θ - ψ(θ)) + y⋅⎜──(ψ(θ)) - 1
⎢                  dθ           ⎝dθ          ⎠                   ⎝dθ          
⎢                                                                             
⎢                  d            ⎛d           ⎞                   ⎛d           
⎢- shank⋅cos(ψ(θ))⋅──(ψ(θ)) - x⋅⎜──(ψ(θ)) - 1⎟⋅cos(θ - ψ(θ)) + y⋅⎜──(ψ(θ)) - 1
⎢                  dθ           ⎝dθ          ⎠                   ⎝dθ          
⎢                                                                             
⎣                                             0                               

⎞              ⎤
⎟⋅cos(θ - ψ(θ))⎥
⎠              ⎥
               ⎥
⎞              ⎥
⎟⋅sin(θ - ψ(θ))⎥
⎠              ⎥
               ⎥
               ⎦

In [ ]:
# As can be seen, the expression can be rewritten 


# from sympy.vector import CoordSys3D
# N = CoordSys3D('N')
# v1 = 2*N.i+3*N.j-N.k
# v2 = N.i-4*N.j+N.k
# v1.dot(v2)
# v1.cross(v2)

v = T_ankle.i * shank

d = T_ankle.rotation_matrix(B) * v


#T_knee.i    
a = d.cross(T_ankle.k)

a
#print(a)

⎛⎡-shank⋅cos(ψ(θ))  shank⋅sin(ψ(θ))     0   ⎤⎞        
⎜⎢                                          ⎥⎟ j_ankle
⎜⎢-shank⋅sin(ψ(θ))  -shank⋅cos(ψ(θ))    0   ⎥⎟        
⎜⎢                                          ⎥⎟        
⎝⎣       0                 0          -shank⎦⎠        

the expression $\frac{\partial \mathbf{p}}{\partial \theta}$ above yields $\frac{\partial x}{\partial\theta}$ and $\frac{\partial y}{\partial\theta}$. $\frac{\partial z}{\partial\theta}$ is $0$.


To obtain the orientation part, we only need to consider the angle $\gamma$ as the joint mechanism is 2D (rotatation around z axis). In other words  $\frac{\partial \alpha}{\partial\theta} = \frac{\partial \beta}{\partial\theta} = 0$



It is

$$ 
\gamma = \gamma_{0} -\psi + \theta
$$

Partial derivative:

$$
\frac{\partial \gamma}{\partial\theta} = 1 - \frac{\partial \psi}{\partial\theta} 
$$




# Automatic code generation

It is advantageous to implement the equations above by hand. Sine and cosine terms can be computed once for improved efficiency.

*The following is just for reference*

In [ ]:
from sympy import ccode #, cpp_generator

def generate_code():

  cpp_code = ccode(dp_dtheta[0], assign_to="dx_dtheta", standard="c11")

  with open("/tmp/expressions.cpp", "w") as f:
    f.write(cpp_code)